In [ ]:
##########################################################################################
#                            Stuff a tener en cuenta                                     #
##########################################################################################
# 1. Data profiling
# 2. Visualizations para outliers
# 3. Datos irrelevantes
# 4. Eliminar duplicados
# 5. Calcular varianza de columnas
# 6. Usar random forest para calcular variables importantes
# 7. Conversión de tipos (a tipos indicados en la documentación en principio)
# 8. Dividir datos complejos
# 9. Arreglar typos
# 10. Feature engineering
# 11. Estandarizar datos
# 12. Eliminar columnas con muchos valores nulos o poca varianza
# 12. Hacer algo con valores null, "" y NaN
# 13. Si hay atributos derivados, comprobar que son correctos
# 14. Hacer algo con datos inválidos de acuerdo a la documentación
# 15. Darle una vuelta a si los datos tienen sentido
# 16. Considerar si tiene sentido considerar datos externos (ej. riesgo de muerte dada una enfermedad)

In [1]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

In [2]:
def read_csvs():
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

In [16]:
dfs = read_csvs()

/tmp/ipykernel_15822/2131931280.py:10: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


In [ ]:
###########################################################################
# Columnas con muchos null (comparativamente. hay otras con 36K y 3K null #
###########################################################################

# allergy (2474)
# 9   drugname              1480 non-null   object 
# 12  drughiclseqno         1480 non-null   float64

# apachePatientResult (3676)
# 20  actualventdays                 1042 non-null   float64
# 21  predventdays                   1004 non-null   float64
# 22  unabridgedactualventdays       1042 non-null   float64

# apachePredVar (2205)
# 27  electivesurgery    342 non-null    float64

# carePlanProvider (5627)
# 3   providertype            0 non-null      float64

# carePlanGoal (3633)
# 4   cplgoalvalue         2128 non-null   object

# carePlanInfectiousDisease (112)
# 6   responsetotherapy        2 non-null      object
# 7   treatment                62 non-null     object

# customLab (30)
# 5   labotherresult     18 non-null     float64

# infusiondrug (38256)
# 5   infusionrate       16779 non-null  float64
# 6   drugamount         13813 non-null  float64
# 7   volumeoffluid      14108 non-null  float64
# 8   patientweight      4036 non-null   float64

# medication (75604)
# 11  loadingdose         8 non-null      object

# microLab (342)
# 5   antibiotic          139 non-null    object
# 6   sensitivitylevel    136 non-null    object

# respiratoryCare (5436)
# 4   airwaytype            794 non-null    object 
# 5   airwaysize            636 non-null    float64
# 6   airwayposition        965 non-null    object 
# 7   cuffpressure          103 non-null    float64
# 12  apneaparams           111 non-null    object 
# 13  lowexhmvlimit         2050 non-null   float64
# 14  hiexhmvlimit          542 non-null    float64
# 15  lowexhtvlimit         551 non-null    float64
# 16  hipeakpreslimit       452 non-null    float64
# 17  lowpeakpreslimit      177 non-null    float64
# 18  hirespratelimit       837 non-null    float64
# 19  lowrespratelimit      286 non-null    float64
# 20  sighpreslimit         0 non-null      float64
# 21  lowironoxlimit        0 non-null      float64
# 22  highironoxlimit       0 non-null      float64
# 23  meanairwaypreslimit   0 non-null      float64
# 24  peeplimit             200 non-null    float64
# 25  cpaplimit             253 non-null    float64
# 26  setapneainterval      255 non-null    float64
# 27  setapneatv            222 non-null    float64
# 28  setapneaippeephigh    23 non-null     float64
# 29  setapnearr            245 non-null    float64
# 30  setapneapeakflow      240 non-null    float64
# 31  setapneainsptime      112 non-null    float64
# 32  setapneaie            0 non-null      float64
# 33  setapneafio2          247 non-null    float64

# vitalAperiodic (274088)
# 6   paop                  169 non-null     float64
# 7   cardiacoutput         14235 non-null   float64
# 8   cardiacinput          11865 non-null   float64
# 9   svr                   17789 non-null   float64
# 10  svri                  3440 non-null    float64
# 11  pvr                   169 non-null     float64
# 12  pvri                  168 non-null     float64

# vitalPeriodic
# 3   temperature        112820 non-null   float64
# 6   respiration        1381351 non-null  float64
# 7   cvp                202284 non-null   float64
# 8   etco2              72780 non-null    float64
# 9   systemicsystolic   226475 non-null   float64
# 10  systemicdiastolic  226471 non-null   float64
# 11  systemicmean       228400 non-null   float64
# 12  pasystolic         32083 non-null    float64
# 13  padiastolic        32080 non-null    float64
# 14  pamean             32632 non-null    float64
# 15  st1                613267 non-null   float64
# 16  st2                656963 non-null   float64
# 17  st3                596372 non-null   float64
# 18  icp                13193 non-null    float64


#--------------------------------------------------------------------------------------------------------------------------------------------------------

#for df_name in dfs:
#    print(df_name + '\n')
#    dfs[df_name].info()
#    print()


# admissionDrug
#dfs['medication']['loadingdose'].value_counts()

In [55]:
dfs['carePlanCareProvider']['providertype'].isna()

0       True
1       True
2       True
3       True
4       True
        ... 
5622    True
5623    True
5624    True
5625    True
5626    True
Name: providertype, Length: 5627, dtype: bool